In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools

path_root = '/home/hwkang/jupyter/root'

import os
import sys
sys.path.append(path_root)
sys.path.append(os.path.join(path_root,'utility/'))
sys.path.append(os.path.join(path_root,'model/'))
sys.path.append(os.path.join(path_root,'experiment/experiment-09/'))
from models import CNN

from utility.statistic import *

In [5]:
class MyNormalize:
    def __init__(self):
        pass
    def __call__(self, data):
        vmax, vmin = data.max(), data.min()
        return (data-vmin)/(vmax-vmin)

In [6]:
def batch_accuracy(train_loader, net):
    with torch.no_grad():
        total = 0
        acc = 0
        net.eval()

        train_loader = iter(train_loader)
        all_labels = []
        all_predictions = []
        for data, targets in train_loader:
            data = data.to(device)
            targets = targets.to(device)
            #spk_rec, _ = forward_pass(net, num_steps, data)
            #spk_rec, _ = net(data)
            spk_rec = net(data)

            #acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
            #total += spk_rec.size(1)

            predicted_value, predicted_class = torch.max(spk_rec, dim=1)
            all_labels.extend(targets.detach().cpu().numpy())
            all_predictions.extend(predicted_class.detach().cpu().numpy())

    acc, _, _, _ = get_classification_metrics(all_labels, all_predictions, 'weighted')
    return acc

In [7]:
transform = transforms.Compose([transforms.ToTensor(), MyNormalize()])
train_dataset = datasets.CIFAR10(root='/home/hwkang/jupyter/root/dataset', train=True, transform=transform, download=False)
test_dataset = datasets.CIFAR10(root='/home/hwkang/jupyter/root/dataset', train=False, transform=transform, download=False)

In [8]:
batch_size=64

In [9]:
### 훈련 데이터셋과 검증 데이터셋으로 분할
dataset_size = len(train_dataset)
train_size = int(0.9 * dataset_size)
valid_size = dataset_size - train_size
train_dataset, valid_dataset = random_split(train_dataset, [train_size, valid_size])
##*

### 데이터로더 준비
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [11]:
#num_steps=50

# Model
#net = Net(num_steps=num_steps, beta=0.9, spike_grad=surrogate.atan())
net = CNN()
net = net.to(device)

# Epoch
num_epochs = 50

# Loss func.
#loss_fn_rate = SF.ce_rate_loss()
criterion = nn.CrossEntropyLoss()

# Optim.
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.999))

# Util.
loss_hist = []
test_acc_hist = []
counter = 0
min_acc = 0.0
best_acc_epoch = 0

In [12]:
# Training loop
for epoch in range(num_epochs):
    for data, targets in iter(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        #spk_rec, _ = forward_pass(net, num_steps, data)
        #spk_rec, _ = net(data)
        spk_rec = net(data)

        # initialize the loss & sum over time
        loss = criterion(spk_rec, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if counter % 100 == 0:
            #test_acc = batch_accuracy(valid_loader, net, num_steps)
            test_acc = batch_accuracy(valid_loader, net)
            print(f'Epoch: {epoch+1}/{num_epochs} | Iter.: {counter} | Acc.: {test_acc*100:.2f}%')
        counter += 1

        if( test_acc > min_acc ):
            best_acc_epoch = epoch
            best_model_state = net.state_dict()

/home/hwkang/jupyter/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch: 1/50 | Iter.: 0 | Acc.: 10.58%
Epoch: 1/50 | Iter.: 100 | Acc.: 41.10%
Epoch: 1/50 | Iter.: 200 | Acc.: 42.90%
Epoch: 1/50 | Iter.: 300 | Acc.: 48.34%
Epoch: 1/50 | Iter.: 400 | Acc.: 52.24%
Epoch: 1/50 | Iter.: 500 | Acc.: 46.90%
Epoch: 1/50 | Iter.: 600 | Acc.: 55.08%
Epoch: 1/50 | Iter.: 700 | Acc.: 51.38%


/home/hwkang/jupyter/venv/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch: 2/50 | Iter.: 800 | Acc.: 54.58%
Epoch: 2/50 | Iter.: 900 | Acc.: 58.44%
Epoch: 2/50 | Iter.: 1000 | Acc.: 57.66%
Epoch: 2/50 | Iter.: 1100 | Acc.: 54.42%
Epoch: 2/50 | Iter.: 1200 | Acc.: 56.60%
Epoch: 2/50 | Iter.: 1300 | Acc.: 56.54%
Epoch: 2/50 | Iter.: 1400 | Acc.: 57.48%
Epoch: 3/50 | Iter.: 1500 | Acc.: 60.00%
Epoch: 3/50 | Iter.: 1600 | Acc.: 59.58%
Epoch: 3/50 | Iter.: 1700 | Acc.: 60.46%
Epoch: 3/50 | Iter.: 1800 | Acc.: 54.46%
Epoch: 3/50 | Iter.: 1900 | Acc.: 62.22%
Epoch: 3/50 | Iter.: 2000 | Acc.: 58.58%
Epoch: 3/50 | Iter.: 2100 | Acc.: 62.22%
Epoch: 4/50 | Iter.: 2200 | Acc.: 61.06%
Epoch: 4/50 | Iter.: 2300 | Acc.: 55.06%
Epoch: 4/50 | Iter.: 2400 | Acc.: 57.02%
Epoch: 4/50 | Iter.: 2500 | Acc.: 62.24%
Epoch: 4/50 | Iter.: 2600 | Acc.: 56.00%
Epoch: 4/50 | Iter.: 2700 | Acc.: 62.80%
Epoch: 4/50 | Iter.: 2800 | Acc.: 63.60%
Epoch: 5/50 | Iter.: 2900 | Acc.: 63.10%
Epoch: 5/50 | Iter.: 3000 | Acc.: 63.06%
Epoch: 5/50 | Iter.: 3100 | Acc.: 62.18%
Epoch: 5/50 | Iter

In [13]:
print(f'Load model from epoch [{best_acc_epoch}]')
net.load_state_dict(best_model_state)

# Test set forward pass
#test_acc = batch_accuracy(test_loader, net, num_steps)
test_acc = batch_accuracy(test_loader, net)
print(f"Test Acc: {test_acc * 100:.2f}%\n")

Load model from epoch [49]
Test Acc: 72.22%

